# Quick Guide to SQLite
This lab is meant to teach the basics of SQLite as well as implementing SQLite into your projects. Please review the slides regarding SQL queries before starting this lab, as many of sections expect you to know SQL. We'll be using pysqlite for this intro:
* SQL (Structured Query Language) is the language used to communicate with the database.
* SQLite is a Relational Database Managment System (DBMS) that, unlike other DBMS, isn't a client-server database engine, meaning we do not need to create a server for it to function.
* Pysqlite is a SQL interface compliant for Python.

**IF YOU GET A DATABASE LOCKED ERROR, TRY:** 
- Restarting the Jupyter Notebook kernel.
- Deleting the `database.db` in the base directory, then repopulating the file.

Run the following function below to repopulate the `database.db` file if necessary.

In [1]:
import database_util
database_util.testDatabase()

## Setup

Make sure the bottom shows the SQLite version.

In [2]:
import sqlite3 as sqlite
import pandas as pd

con = None
data = 'Not found'

try:
    con = sqlite.connect(":memory:")
    cur = con.cursor()
    cur.execute('SELECT SQLITE_VERSION()')
    data = cur.fetchone()[0]

except sqlite.Error as e:
    print(e)

finally:
    if con:
        con.close()
print("SQLite version: {}".format(data))

SQLite version: 3.22.0


# Connections and Cursors

When we want to access our database, we must create a `connection` object to the database file in question and obtain a `cursor` from that connection object.
* `Connection`: An object that allows access to the database for reading and writing.
* `Cursor`: An object that is mainly used to execute queries on the database.

The process of querying a database would often go like this

1. A `connection` to the database is opened using `sqlite3.connect("<name of database file>")`. 
    - If the database file doesn't exist, that file is automatically created.
    - The name of the database file can be `":memory:"` to access an in-memory database, which is a __distinct__ database for every connection.
2. A `cursor` is obtained from the connection `con` using `con.cursor()`.
3. The user executes queries with the cursor.
4. If the user writes to the database, the connection `con` must `con.commit()` to save changes for other connections to view (more on that later).

Connections are garbage collected overtime, but it is a good practice to `close()` the connection when you are finished executing all the queries.

### PreQ1: Opening a new connection
Define a helper function `open_new_connection` that takes in a String `database`, then creates and returns a new `connection` and its `cursor`.

In [3]:
def open_new_connection(database):
    # DO NOT CHANGE ANY PART OF THIS CODE UNLESS STATED OTHERWISE
    con = sqlite.connect(database) # EDIT THIS LINE
    cur = con.cursor() # EDIT THIS LINE
    
    return con, cur

# Querying The Database

When you want to execute a query, use `execute()` on a cursor and input your query in string format. We'll be reviewing `execute()` in the next sections, but before moving on however, please review the following terms:
* `Table`: A relation that contains a `Schema` and `Entries`
* `Schema`: A set of rules that the table follows (i.e., INT id)
* `Entry`: A tuple that follows the `Schema`
    
For example, if we define a table `People` with schema: `(id INT, FirstName TEXT, LastName TEXT, age INT)`, the table would look something like this:


In [4]:
con = sqlite.connect(":memory:")
cur = con.cursor()

schema = ['id INT', 'FirstName TEXT', 'LastName TEXT', 'age INT']
cur.execute("CREATE TABLE ExamplePeople ('id INT', 'FirstName TEXT', 'LastName TEXT', 'age INT')")
queries = [(2133, 'Elton', 'John', 72), 
           (76, 'Ana', 'Amari', 2), 
           (69, 'Elon', 'Musk', 47), 
           (123, 'Count von', 'Count', 46)
          ]
cur.executemany("INSERT INTO ExamplePeople VALUES (?,?,?,?)", queries)
cur.execute("SELECT * FROM ExamplePeople")
p = pd.DataFrame(cur.fetchall(), columns=schema)

cur.execute("DROP TABLE IF EXISTS ExamplePeople")
con.close()
p

,id INT,FirstName TEXT,LastName TEXT,age INT
0,2133,Elton,John,72
1,76,Ana,Amari,2
2,69,Elon,Musk,47
3,123,Count von,Count,46


For SQLite, the `schema` can include variables of the following five types (with their associated pythonic types):
* `NULL`: None
* `INTEGER`: Int or Long
* `REAL`: Float
* `TEXT`: String
* `BLOB`: [Buffer](https://docs.python.org/3/c-api/buffer.html)

## Execute


Pysqlite accesses SQLite databases through `execute()`, which allows Python to execute SQL commands in string format. This makes it extrememly useful to fetch and manipulate data in databases through Python scripts and functions. 

To run it,`execute()` must be called using an open `cursor` object as well as a SQL string query within it. For example: 

`cur.execute("CREATE TABLE People (id INT, name TEXT))`

### Parameter substitution
`execute()` supports parameter substitution by using placeholders, which can be in the form of a string dictionary keys or question marks. This has uses with accepting user input as part of the query and as a good way to protect against SQL Injection. An example is shown below:

In [5]:
con = sqlite.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE Paintings (id INT, price INT)")

painting_id = 20
painting_cost = 20000

# Example of using parameter substitution using questionmarks
cur.execute("INSERT INTO Paintings VALUES (?, ?)", (painting_id, painting_cost))

# Example of using a dictionary
cur.execute("SELECT * FROM Paintings WHERE id=:id AND price=:cost", {"id" : painting_id, "cost" : painting_cost})
print(cur.fetchone())
con.close()

(20, 20000)


### executemany()
`executemany()` allows you to execute multiple execute statements using parameter substitution by taking in a list of tuples rather than a single tuple. This is mainly used for running multiple INSERT commands.

In [6]:
con = sqlite.connect(":memory:")
cur = con.cursor()

cur.execute("CREATE TABLE Transactions (id INT, money INT)")

transactions = [
    (23, 200),
    (76, 800),
    (999, 22),
    (12, 34)
]

# Execute many example using parameter substitution
cur.executemany("INSERT INTO Transactions VALUES (?, ?)", transactions)
cur.execute("SELECT * FROM Transactions ORDER BY id ASC")

print(cur.fetchall())
con.close()

[(12, 34), (23, 200), (76, 800), (999, 22)]


## Fetching Data
To access data from the database:
* `execute()` a SELECT command using a Cursor object `cur`.
* Fetch from the cursor using either `fetchone()` or `fetchall()`
    * `fetchone()` only returns a single row (in tuple form)
    * `fetchall()` returns all the rows from the command using a list of tuples.

The fetch function must happen right after the `SELECT` command on the cursor.

In [7]:
con = sqlite.connect(":memory:")
cur = con.cursor()

queries = [
    (1, '2'),
    (3, '4'),
    (5, '6')
]

cur.execute("CREATE TABLE hello (id, str)")
cur.executemany("INSERT INTO hello VALUES (?, ?)", queries);

# Fetching one entry from the table
cur.execute("SELECT * FROM hello")
print("Fetching one       : {}".format(cur.fetchone()))
print("Fetching one again : {}".format(cur.fetchone()))

# Fetching all the entries from the table
cur.execute("SELECT * FROM hello")
print("Fetching all       : {}".format(cur.fetchall()))
con.close()

Fetching one       : (1, '2')
Fetching one again : (3, '4')
Fetching all       : [(1, '2'), (3, '4'), (5, '6')]


## Changing the data
Whenever an insert is made to a database, you must commit that change with the connection so that the change is visible to other connections. 
* It's effectively useless to commit on an in-memory database (i.e. `:memory:`) as each connection connects to a __distinct__ database, so a connection will never see another connection's changes.

Below is an demonstration of committing and what other connections can see.

In [8]:
# Creating two different connections as a demonstration
con = sqlite.connect('database.db')
cur = con.cursor()

diff_con = sqlite.connect('database.db')
diff_cur = diff_con.cursor()

# SQL queries
cur.execute("DROP TABLE IF EXISTS example")
cur.execute("CREATE TABLE example (id INT)")
cur.execute("INSERT INTO example VALUES (1)")

# Noncommitted change
diff_cur.execute("SELECT * FROM example")
print("Before commit: {}".format(diff_cur.fetchall()))

# Committed change
con.commit()
diff_cur.execute("SELECT * FROM example")
print("After commit : {}".format(diff_cur.fetchall()))
diff_con.close()
con.close()

Before commit: []
After commit : [(1,)]


### PreQ2: Committing a change
The function `hello_world()` below writes to database `database.db` a table that takes in an integer `id` and string `msg` field and creates an entry with `id` being 0 and `string` being 'Hello'. However, the function is not quite complete. Add one additional line to the function so that it executes correctly.

In [9]:
def hello_world():
    # DO NOT CHANGE ANY PART OF THIS CODE UNLESS STATED OTHERWISE
    con, cur = open_new_connection("database.db")
    
    cur.execute("DROP TABLE IF EXISTS q2")
    cur.execute("CREATE TABLE q2(id INT, msg TEXT)")
    cur.execute("INSERT INTO q2 VALUES(0, 'Hello')")
    
    # ADD A LINE HERE:
    con.commit()
    
hello_world()
con, cur = open_new_connection("database.db")
cur.execute("SELECT * FROM q2")
cur.fetchall()

[(0, 'Hello')]

## Additional tips and tricks

### Helper functions
It may be helpful to create helper functions on SQL commands when working on your projects.

In [10]:
def create_connection(db_file):
    """
    Creates a connection to the database specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    con = None
    try:
        con = sqlite.connect(db_file)
    except sqlite.Error as e:
        print("Error {}:".format(e.args[0]))
    
    return con

def search_for_ids(con, id):
    """
    Uses a connection object to search for entries with id's equal to id.
    Prints all the ids
    :param con: connection object, :param id: integer
    """
    cur = con.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS test (id INT)")
    
    # Avoids SQL injection
    cur.execute("SELECT * FROM test WHERE id =?", (id,))
    data = cur.fetchall()
    
    return data

con = create_connection(":memory:")
con.execute("CREATE TABLE test (id INT)")
con.execute("INSERT INTO test VALUES (5)")
print(search_for_ids(con, 5))
con.close()

[(5,)]


### Pandas and SQLite

Pandas also can interact with SQLite databases using the `to_sql` and `read_sql` functions.
* `to_sql` is executed on a `Dataframe` and takes in the `name` of a table as well as a SQLite connection `con`. This will create a `name` table within `con`.
* `read_sql` is executed on `pandas` and takes in a SQL query on a table `name` as well as a SQLite connection `con`. This will create a `Dataframe` with `name` table's contents.

### pandas.Dataframe.to_sql()

In [11]:
data = [['Watson', 1], ['Holmes', 2], ['Stamford', 3]]

df = pd.DataFrame(data, columns=['name', 'id'])

con = sqlite.connect('database.db')
cur = con.cursor()

con.execute("DROP TABLE IF EXISTS NAMES")
df.to_sql('Names', con)

print(cur.execute("SELECT * FROM NAMES").fetchall())
con.close()

[(0, 'Watson', 1), (1, 'Holmes', 2), (2, 'Stamford', 3)]


### pandas.read_sql()

In [12]:
con = sqlite.connect('database.db')
df = pd.read_sql("SELECT * FROM NAMES WHERE id < 3", con)
con.close()
df

,index,name,id
0,0,Watson,1
1,1,Holmes,2


## Questions

All of the questions will assume that you have completed all of the top sections __AND__ have an understanding of the SQL language (meaning you have reviewed the [lecture notes](https://docs.google.com/presentation/d/1UYgX5zewhPOvR2I9TlWPbz0tysjGMncwRb8a7EEnxxg/edit?usp=sharing)). All functions must also not cause any errors such as locking the database after execution.

**IF YOU GET A DATABASE LOCKED ERROR, TRY:** 
- Restarting the Jupyter Notebook kernel.
- Deleting the `database.db` in the base directory.

### Q1: Creating a table
Define the function `create_table()` below to create a table in the `database.db` file named `ids` which contains the columns `id INTEGER, name TEXT`.

- The function should not error after multiple calls.
- If the `ids` table exists, the function should not change the database file.

In [13]:
def create_table():
    con, cur = open_new_connection("database.db")
    
    cur.execute('''CREATE TABLE IF NOT EXISTS ids (
                    id INTEGER PRIMARY KEY,
                    name TEXT
                )''')
    con.commit()
    con.close()

### Q2: Setting up the table
Define the function `populate_table()` below to populate a table named `ids` which contains the columns `id INTEGER, name TEXT` with the following values
`[(1, 'Oski'), (2, 'Is'), (3, 'The'), (4, 'Best')`

- Regardless of database state, after the function is called, the database must contain a table named `ids` with only the stated columns and only the stated values.

In [14]:
def populate_table():
    con, cur = open_new_connection("database.db")
    cur.execute("DROP TABLE IF EXISTS ids")
    create_table()
    values = [
        (1, 'Oski'),
        (2, 'Is'),
        (3, 'The'),
        (4, 'Best')
    ]
    cur.executemany("INSERT INTO ids VALUES (?, ?)", values)
    con.commit()
    con.close()
populate_table()
con, cur = open_new_connection("database.db")
cur.execute("SELECT * FROM ids")
cur.fetchall()

[(1, 'Oski'), (2, 'Is'), (3, 'The'), (4, 'Best')]

You are a manager at Oski Corp and you have two tables, the first table is a list of `Employees` while the second table is a list of `Careers`, as depicted below. Use your knowledge of SQL to answer the questions below!

You may assume that these tables already placed in the `database.db`. Assume no fields are NULL. 

Table Employees:
* `EmployeeID`: Integer, Primary Key
* `FirstName`: String
* `LastName`: String

Table Careers:
* `EmployeeID`: Integer, Primary Key, references Employees
* `CareerID`: Integer, Primary Key
* `Field`: String
* `Salary`: Float

The test tables look something like this:

Table `Employees`

| EmployeeID | FirstName | LastName |
|------------|-----------|----------|
| 0          | Harry     | Truman   |
| 1          | John      | Doe      |
| 2          | Rachel    | Green    |
| 3          | Ross      | Geller   |
| 4          | John      | Cena     |
| 5          | Jonathan  | Joestar  |
| 6          | Sherlock  | Holmes   |
| 7          | John      | Watson   |

Table `Careers`

| EmployeeId | CareerId | Field       | Salary |
|------------|----------|-------------|--------|
| 0          | 7        | Engineering | 17.10  |
| 1          | 6        | Sales       | 20.50  |
| 2          | 5        | HR          | 50.00  |
| 3          | 4        | IT          | 49.99  |
| 4          | 3        | Sales       | 100.01 |
| 5          | 2        | IT          | 2.00   |
| 6          | 1        | HR          | 19.50  |
| 7          | 0        | Investments | 39.00  |

Run the following function below to repopulate the `database.db` file if necessary.

In [15]:
import database_util
database_util.testDatabase()

### Q3: My name is John
Define the function `find_john()` below to find all `Employees` with the `FirstName` "John". 
- Return a list of `EmployeeID`, `FirstName`, and `LastName` tuples sorted by `EmployeeID` in ascending order.

In [16]:
def find_john():
    con, cur = open_new_connection("database.db")
    
    # DO NOT EDIT CODE ABOVE THIS POINT
    cur.execute("""
        SELECT * 
        FROM Employees 
        WHERE FirstName='John' 
        ORDER BY EmployeeID ASC""")
    output = cur.fetchall()
    con.close()
    return output
    # DO NOT EDIT CODE BELOW THIS POINT
find_john()

[(1, 'John', 'Doe'), (4, 'John', 'Cena'), (7, 'John', 'Watson')]

### Q4 Field Popularity

Define the function `field_popularity()` below to return a list of the number of people employed in each `field`, sorted in ascending order by the number of people, then in ascending order by field.

In [17]:
def field_popularity():
    con, cur = open_new_connection("database.db")
    cur.execute("""
        SELECT COUNT(*), field 
        FROM Careers 
        GROUP BY field 
        ORDER BY COUNT(*) ASC, field ASC
    """)
    output = cur.fetchall()
    con.close()
    return output
field_popularity()

[(1, 'Engineering'), (1, 'Investments'), (2, 'HR'), (2, 'IT'), (2, 'Sales')]

### Q5 Max Salary titles
Finish the function below to find the `Career` with the largest `Salary` for every `Field`.  Return a list of tuples containing:
* `EmployeeID`, `FirstName`, and `LastName` of the `Employee` with the largest `Salary` in their `Field`.
* The `Field` in question
* The maximum `Salary` in that specific `Field`

List also must be sorted in descending order based on `salary`.

In [18]:
def max_salary_title():
    con, cur = open_new_connection("database.db")
    cur.execute('''
                SELECT a.EmployeeID, a.FirstName, a.LastName, b.Field, MAX(b.Salary)
                FROM Employees AS a
                INNER JOIN Careers AS b 
                ON a.EmployeeID=b.EmployeeID
                GROUP BY b.Field
                ORDER BY MAX(b.Salary) DESC
                ''')
    output = cur.fetchall()
    con.close()
    return output
max_salary_title()

[(4, 'John', 'Cena', 'Sales', 100.01),
 (2, 'Rachel', 'Green', 'HR', 50.0),
 (3, 'Ross', 'Geller', 'IT', 49.99),
 (7, 'John', 'Watson', 'Investments', 39.0),
 (0, 'Harry', 'Truman', 'Engineering', 17.1)]

Your boss provides you a `EmployeeBosses` table and asks you to combine it with the `Employees` table in a **Full Outer Join**. She wants you to create a **DataFrame** with the following columns:

- Distinct `EmployeeIDs` from both the `Employees` and `EmployeeBoss` tables
- `FirstName` from `Employees` if exists, else `NULL`.
- `LastName` from `Employees` if exists, else `NULL`.
- `BossFirstName` from `EmployeeBosses` if exists, else `NULL`.
- `BossLastName` from `EmployeeBosses` if exists, else `NULL`.

The `EmployeeBosses` table is composed of:

- An `EmployeeID`: Integer, Primary Key
- `BossFirstName`: String
- `BossLastName`: String

There may be some `EmployeeIDs` that only exist in `Employees` and some `EmployeeIDs` that only exist in `EmployeeBosses`.

Table `EmployeeBosses`:

|EmployeeID|BossFirstName|BossLastName|
|----------|-------------|------------|
|4         |Cant C.      |Mei         |
|5         |Will         |Zeppeli     |
|6         |Inspector    |Lestrade    |
|7         |Sherlock     |Holmes      |
|8         |Oski         |Bear        |
|9         |John         |Hammond     |
|10        |Richard      |Feynman     |
|11        |George       |Lucas       |

### Q6a Full Outer Join
You realize from the lecture that SQLite doesn't support Full Outer Joins and relies on a [certain workaround](https://en.wikipedia.org/wiki/Join_%28SQL%29#Full_outer_join). Define a function to return the **STRING QUERY** of that workaround of the Full Outer Join stated above. The query must sort on `EmployeeID` in ascending order.

In [19]:
def full_outer_join():
    return '''
                SELECT a.EmployeeID, a.FirstName, a.LastName, b.BossFirstName, b.BossLastName
                FROM Employees as a 
                LEFT JOIN EmployeeBosses as b 
                ON a.EmployeeId = b.EmployeeId
                UNION ALL
                SELECT a.EmployeeID, b.FirstName, b.LastName, a.BossFirstName, a.BossLastName
                FROM EmployeeBosses as a
                LEFT JOIN Employees as b
                ON a.EmployeeId = b.EmployeeId
                WHERE b.EmployeeId IS NULL
                '''

### Q6b Dataframe
Using `pandas`, define the function `full_outer_join_df()` to create and return a `DataFrame` based on the SQL query that you wrote in Q6a.

In [20]:
def full_outer_join_df():
    con = sqlite.connect('database.db')
    return pd.read_sql(full_outer_join(), con)
full_outer_join_df()

,EmployeeID,FirstName,LastName,BossFirstName,BossLastName
0,0,Harry,Truman,None,None
1,1,John,Doe,None,None
2,2,Rachel,Green,None,None
3,3,Ross,Geller,None,None
4,4,John,Cena,Cant C.,Mei
5,5,Jonathan,Joestar,Will,Zeppeli
6,6,Sherlock,Holmes,Inspector,Lestrade
7,7,John,Watson,Sherlock,Holmes
8,8,None,None,Oski,Bear
9,9,None,None,John,Hammond


## Submitting

At this point, you may submit the Jupyter Notebook file `SQLLab.ipynb` directly to Gradescope under the assignment name INSERT ASSIGNMENT NAME HERE. **MAKE SURE ALL CELLS RUN CORRECTLY AND WITHOUT ERRORS, ELSE THE AUTOGRADER MAY NOT FUNCTION PROPERLY.**
The autograder will be grading all the pre-questions as well as all the questions.

Functions needed:

- PQ1: `open_new_connection()`
- PQ2: `hello_world()`
- Q1: `create_table()`
- Q2: `populate_table()`
- Q3: `find_john()`
- Q4: `field_popularity()`
- Q5: `max_salary_title()`
- Q6a: `full_outer_join()`
- Q6b: `full_outer_join_df()`